In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [2]:
# Reading data
df_origin = pd.read_csv(r"C:\Users\s\Downloads\destinyArmor.csv")
df = df_origin.copy()

# Filtering out unwanted gear
df = df[
    (df["Type"] != "Titan Mark") &
    (df["Total (Base)"] > 60) &
    (df["Tier"] != "Rare") &
    (df["Power Limit"] != 1060)
]

def process_gear(input_df, sort_tier_by="Total", masterwork=False):
            
    # Error handling for sort_tier_by param
    acceptable_sort_tier_by = ["Mobility", "Resilience", "Recovery", "Discipline", "Intellect", "Strength", "Total"]
    error_msg = "sort_tier_by must be string or list containing: Mobility, Resilience, Recovery, Discipline, Intellect, Strength or Total"
    if type(sort_tier_by) is str:
        if sort_tier_by not in acceptable_sort_tier_by:
            raise ValueError(error_msg)
    elif type(sort_tier_by) is list:
        unacceptable_items = [item for item in sort_tier_by if item not in acceptable_sort_tier_by]
        if len(unacceptable_items) > 0:
            raise ValueError("The following don't match 'Mobility', 'Resilience', 'Recovery', 'Discipline', 'Intellect', 'Strength' or 'Total': ", unacceptable_items)
    else:
        raise ValueError(error_msg)

    # Setting empty dataframe
    output_df = pd.DataFrame(columns=["Index", "Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"])
    
    # Setting up a group for each armour combination, inserting them into output_df
    for i in range(len(input_df.loc[input_df["Type"] == "Helmet"])):
        tmp_df = pd.DataFrame(columns=["Index", "Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"])

        df_helm = input_df.loc[input_df["Type"] == "Helmet", ["Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"]].reset_index().drop('index', axis=1)
        tmp_df = tmp_df.append(df_helm.loc[i, :])
        print(1)
        
        for j in range(len(input_df.loc[input_df["Type"] == "Gauntlets"])):

            if len(tmp_df[tmp_df['Type'] == 'Gauntlets']) > 0:
                tmp_df = tmp_df[tmp_df.Type != 'Gauntlets']

            df_gauntlets = input_df.loc[input_df["Type"] == "Gauntlets", ["Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"]].reset_index().drop('index', axis=1)
            tmp_df = tmp_df.append(df_gauntlets.loc[j, :])

            if len(output_df) == 0:
                tmp_df['Index'] = 0
            else:
                tmp_df['Index'] = output_df['Index'].max() + 1
                
            for k in range(len(input_df.loc[input_df["Type"] == "Chest Armor"])):
                if len(tmp_df[tmp_df['Type'] == 'Chest Armor']) > 0:
                    tmp_df = tmp_df[tmp_df.Type != 'Chest Armor']

                df_chest = input_df.loc[input_df["Type"] == "Chest Armor", ["Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"]].reset_index().drop('index', axis=1)
                tmp_df = tmp_df.append(df_chest.loc[k, :])

                if len(output_df) == 0:
                    tmp_df['Index'] = 0
                else:
                    tmp_df['Index'] = output_df['Index'].max() + 1
                
                for l in range(len(input_df.loc[input_df["Type"] == "Leg Armor"])):
                    if len(tmp_df[tmp_df['Type'] == 'Leg Armor']) > 0:
                        tmp_df = tmp_df[tmp_df.Type != 'Leg Armor']

                    df_legs = input_df.loc[input_df["Type"] == "Leg Armor", ["Type", "Id", "Name", "Power", "Mobility (Base)", "Resilience (Base)", "Recovery (Base)", "Discipline (Base)", "Intellect (Base)", "Strength (Base)", "Total (Base)"]].reset_index().drop('index', axis=1)
                    tmp_df = tmp_df.append(df_legs.loc[l, :])

                    if len(output_df) == 0:
                        tmp_df['Index'] = 0
                    else:
                        tmp_df['Index'] = output_df['Index'].max() + 1

                    output_df = output_df.append(tmp_df)

    # Renaming columns
    output_df = output_df.rename(columns={"Mobility (Base)": "Mobility", "Resilience (Base)": "Resilience", "Recovery (Base)": "Recovery", "Discipline (Base)": "Discipline", "Intellect (Base)": "Intellect", "Strength (Base)": "Strength", "Total (Base)": "Total"})

    # Adds 2 to every stat
    if type(masterwork) == bool:
        if masterwork == True:
            output_df['Mobility'] = output_df['Mobility'] + 2
            output_df['Resilience'] = output_df['Resilience'] + 2
            output_df['Recovery'] = output_df['Recovery'] + 2
            output_df['Discipline'] = output_df['Discipline'] + 2
            output_df['Intellect'] = output_df['Intellect'] + 2
            output_df['Strength'] = output_df['Strength'] + 2
            output_df['Total'] = output_df['Mobility'] + output_df['Resilience'] + output_df['Recovery'] + output_df['Discipline'] + output_df['Intellect'] + output_df['Strength']
            
    elif masterwork in ["Helmet", "Gauntlets", "Chest Armor", "Leg Armor"]:
        output_df["Mobility"] = np.where((output_df["Type"] == masterwork), output_df["Mobility"] + 2, output_df["Mobility"])
        output_df["Resilience"] = np.where((output_df["Type"] == masterwork), output_df["Resilience"] + 2, output_df["Resilience"])
        output_df["Recovery"] = np.where((output_df["Type"] == masterwork), output_df["Recovery"] + 2, output_df["Recovery"])
        output_df["Discipline"] = np.where((output_df["Type"] == masterwork), output_df["Discipline"] + 2, output_df["Discipline"])
        output_df["Intellect"] = np.where((output_df["Type"] == masterwork), output_df["Intellect"] + 2, output_df["Intellect"])
        output_df["Strength"] = np.where((output_df["Type"] == masterwork), output_df["Strength"] + 2, output_df["Strength"])
    else:
        print("Unknown masterwork value")  #TOOD: error handling
        
    # "Set" row
    set_df = output_df.groupby('Index').aggregate('sum').reset_index()
    set_df.loc[:, 'Id'] = np.nan
    set_df.loc[:, 'Name'] = np.nan
    set_df.loc[:, 'Power'] = np.nan
    set_df.loc[:, 'Type'] = "Set"

    # "Tier" row
    tier_df = output_df.groupby('Index').aggregate('sum').reset_index()
    tier_df.loc[:, 'Id'] = np.nan
    tier_df.loc[:, 'Name'] = np.nan
    tier_df.loc[:, 'Power'] = np.nan
    tier_df.loc[:, 'Type'] = "Tier"
    tier_df['Mobility'] = (( (tier_df['Mobility'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Resilience'] = (( (tier_df['Resilience'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Recovery'] = (( (tier_df['Recovery'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Discipline'] = (( (tier_df['Discipline'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Intellect'] = (( (tier_df['Intellect'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Strength'] = (( (tier_df['Strength'] / 10).apply(np.floor).astype(int) * 10 ) / 10).astype(int)
    tier_df['Total'] = tier_df['Mobility'] + tier_df['Resilience'] + tier_df['Recovery'] + tier_df['Discipline'] + tier_df['Intellect'] + tier_df['Strength']
    print(2)
    
    # Insert "Set" and "Tier" rows
    extended_df = pd.concat((output_df, set_df))
    extended_df = pd.concat((extended_df, tier_df))
    extended_df = extended_df.reset_index(drop=True)
    
    # Setting order in Type column
    type_order = ['Helmet', 'Gauntlets', 'Chest Armor', 'Leg Armor', 'Set', 'Tier']
    extended_df['Type'] = pd.Categorical(extended_df['Type'], categories=type_order, ordered=True)
    extended_df = extended_df.sort_values('Type')
    extended_df.reset_index(drop=True)
    print(3)
    
    # Ordering each armour set based on tier row
    extended_df['idx'] = extended_df.index  # capture the original index of each row, will be used for sorting later
    tier = extended_df.loc[extended_df['Type']=='Tier']  # create a dataframe with only items that match 'Tier'
    tier = tier.sort_values(sort_tier_by, ascending=False)  # sort for only the Tier rows
    tier_list = tier['Index'].tolist()  # create a list of the indexes in sorted order, will be order to print rows
    sorter_index = dict(zip(tier_list, range(len(tier_list))))  # dict that defines order for sorting
    extended_df['tier_rank'] = extended_df['Index'].map(sorter_index) + 1  # generate rank col that'll be used to sort dataframe numerically
    extended_df.sort_values(['tier_rank','idx'], ascending = [True, True], inplace = True)  # sort dataframe based on rank col and original index
    extended_df = extended_df.drop(['Index','idx'], 1).rename(columns={"tier_rank": "Index"})  # drop temporary col, replacing Index with tier_rank
    extended_df = extended_df.set_index(['Index', 'Type'], drop=True)
    print(4)
    
    return extended_df

# Helm of Saint

In [3]:
# INPUT DF: FILTER THE GEAR!: 
# e.g. Solstice gear, less Solstice helmet, plus helm of Saint-14
#input_df = df[ ((df["Event"] == "Solstice of Heroes") & (df['Name'] != "Solstice Helm (Magnificent)")) | (df["Name"] == "Helm of Saint-14") ]

input_df = df[~( (df['Type'] == "Helmet") & (df["Name"] != "Helm of Saint-14"))]
input_df = input_df[ ~( (input_df['Tier'] == "Exotic") & (input_df["Name"] != "Helm of Saint-14") ) ]

saint14_mw = process_gear(input_df, sort_tier_by=["Total"], masterwork="Helmet")
# saint14_mw.head(720)

1
1
2
3
4


In [ ]:
saint14_mw.to_excel('saint14_mw.xlsx')

In [5]:
saint14_mw.head(100)

Id                              Name  \
Index Type                                                                   
1     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529204042091961"                 Gauntlets of Rull   
      Chest Armor  "6917529203332263007"                   Chassis of Rull   
      Leg Armor    "6917529199555520948"                   Greaves of Rull   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
2     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529203905512979"                 Gauntlets of Rull   
      Chest Armor  "6917529203898146218"                   Chassis of Rull   
      Leg Armor    "6917529203616907246"                  Holdfast Greaves   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
3     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529198855794136"               Devastation Complex   
      Chest Armor  "6917529197502183774"          Insight Unyielding Plate   
      Leg Armor    "6917529199560944295"                   Greaves of Rull   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
4     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529198855794136"               Devastation Complex   
      Chest Armor  "6917529197502183774"          Insight Unyielding Plate   
      Leg Armor    "6917529203616907246"                  Holdfast Greaves   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
5     Helmet       "6917529202217543309"                  Helm of Saint-14   
      Gauntlets    "6917529203611399897"                Holdfast Gauntlets   
      Chest Armor  "6917529203611398520"                    Holdfast Plate   
      Leg Armor    "6917529203616907246"                  Holdfast Greaves   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
6     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529203905512979"                 Gauntlets of Rull   
      Chest Armor  "6917529203611398520"                    Holdfast Plate   
      Leg Armor    "6917529203616907246"                  Holdfast Greaves   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
7     Helmet       "6917529202217543309"                  Helm of Saint-14   
      Gauntlets    "6917529198855794136"               Devastation Complex   
      Chest Armor  "6917529197502183774"          Insight Unyielding Plate   
      Leg Armor    "6917529199555520948"                   Greaves of Rull   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
8     Helmet       "6917529202217543309"                  Helm of Saint-14   
      Gauntlets    "6917529203905512979"                 Gauntlets of Rull   
      Chest Armor  "6917529203965426998"               Moonfang-X7 Chassis   
      Leg Armor    "6917529199555520948"                   Greaves of Rull   
      Set                            NaN                               NaN   
      Tier                           NaN                               NaN   
9     Helmet       "6917529202223945870"                  Helm of Saint-14   
      Gauntlets    "6917529203012523858"             Moonfan

# Dunemarchers

In [ ]:
input_df = df[~( (df['Type'] == "Leg Armor") & (df["Name"] != "Dunemarchers"))]
input_df = input_df[ ~( (input_df['Tier'] == "Exotic") & (input_df["Name"] != "Dunemarchers") ) ]

dunemarchers_mw = process_gear(input_df, sort_tier_by=["Total"], masterwork=True)
dunemarchers_mw.head(720)

In [ ]:
dunemarchers_mw.to_excel('dunemarchers_mw.xlsx')